This Notebook will be used of used for the capstone project

# Introduction

#### For the capstone project, Ill be will be using the clustering algorithm to be able to cluster different neighborhoods in Philadelphia by the similarities they have to each to each other. That way people will be able to explore new parts of the city that are either similar or different to what they are currently used to.

# Data 

#### For the data I will be getting the names of the neighborhoods from wikipedia and doing some data cleaning. Then I will be using those name to find the coordinates of those neighborhoods using google maps api. Finally, by using four square, I will b able to get businesses that are nearby each of the neighborhoods given a radius.

In [104]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv
import re
import json

In [20]:
source = requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Philadelphia').text 
soup = BeautifulSoup(source, 'lxml')

In [22]:
csv_file = open('philadelphia.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Neighbourhood'])

15

In [27]:
mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg) # Gets the length of number of `mw-category-groups` present

for i in range(1, length):  # Gets all the neighbourhoods
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') # Gets the title of the neighbourhood
        csv_writer.writerow([nbd]) # Writes the name of the neighbourhood in the csv file

In [29]:
csv_file.close()

In [30]:
df = pd.read_csv('philadelphia.csv')

In [33]:
df['Neighbourhood']

0      Category:Bridesburg-Kensington-Richmond, Phila...
1                      Category:Bridesburg, Philadelphia
2                      Category:Callowhill, Philadelphia
3                   Category:Chestnut Hill, Philadelphia
4                       Category:Chinatown, Philadelphia
                             ...                        
451                                 Wister, Philadelphia
452                       Woodland Terrace, Philadelphia
453                     Wynnefield Heights, Philadelphia
454                             Wynnefield, Philadelphia
455                               Yorktown, Philadelphia
Name: Neighbourhood, Length: 456, dtype: object

In [34]:
df.shape

(456, 1)

In [57]:
list1 = []
list2 = []
for i in range(len(df)):
    list1.append(bool(re.search('Category:', df['Neighbourhood'][i])))
    list2.append(bool(re.search(', Philadelphia', df['Neighbourhood'][i])))

In [58]:
list2

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 T

In [56]:
df['Neighbourhood'][25]

"Category:Penn's Landing"

In [61]:
temp_df = []
for i in range(len(df)):
    temp_df.append(df['Neighbourhood'][i].split(':'))

In [83]:
temp_df2 = []
for i in range(len(temp_df)):
    if (temp_df[i][0] == 'Category'):
        temp_df2.append(temp_df[i][1])
    else:
        temp_df2.append(temp_df[i][0])
    

In [87]:
temp_df3 = []
for i in range(len(temp_df2)):
    temp_df3.append(temp_df2[i].split(', Philadelphia'))

In [91]:
temp_df4 = []
for i in range(len(temp_df3)):
    temp_df4.append(temp_df3[i][0])

In [98]:
new_df = pd.DataFrame(temp_df4)

In [111]:
new_df.columns = ['Neighborhood']

In [112]:
new_df

,Neighborhood
0,Bridesburg-Kensington-Richmond
1,Bridesburg
2,Callowhill
3,Chestnut Hill
4,Chinatown
...,...
451,Wister
452,Woodland Terrace
453,Wynnefield Heights
454,Wynnefield


In [114]:
API_KEY = '...'

In [116]:
latitudes = []
longitudes = []

for i in new_df['Neighborhood']:
    place_name = i + ",Philadelphia,Unitied States"
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(place_name, API_KEY)
    obj = json.loads(requests.get(url).text)
    
    results = obj['results']
    lat = results[0]['geometry']['location']['lat']
    lng = results[0]['geometry']['location']['lng']
    
    latitudes.append(lat)
    longitudes.append(lng)

In [119]:
len(latitudes)

456

In [122]:
new_df['latitude'] = latitudes
new_df['longitude'] = longitudes

In [123]:
new_df

,Neighborhood,latitude,longitude
0,Bridesburg-Kensington-Richmond,39.999581,-75.069072
1,Bridesburg,39.999581,-75.069072
2,Callowhill,39.959507,-75.155997
3,Chestnut Hill,40.070333,-75.207064
4,Chinatown,39.955205,-75.154278
...,...,...,...
451,Wister,40.035112,-75.157955
452,Woodland Terrace,39.948801,-75.204533
453,Wynnefield Heights,40.002128,-75.211129
454,Wynnefield,39.989835,-75.237681


In [126]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/jordankupec/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.9.0                |   py37h2987424_1         3.0 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.7             |          1_cp37m

In [140]:
CLIENT_ID = '...'
CLIENT_SECRET = '...'
VERSION = '20201028'


In [130]:
# Philadelphia latitude and longitude using Google search
phil_lat = 39.9526
phil_lng = -75.1652

# Creates map of Kolkata using latitude and longitude values
map_philadelphia = folium.Map(location=[phil_lat, phil_lng], zoom_start=10)

# Add markers to map
for lat, lng, neighbourhood in zip(new_df['latitude'], new_df['longitude'], new_df['Neighborhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_philadelphia)  
    
map_philadelphia

In [131]:
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [146]:
explore_df_list = []

for i, nbd_name in enumerate(df['Neighbourhood']):  
    
    try :
        ### Getting the data of neighbourhood
        nbd_name = new_df.loc[i, 'Neighborhood']
        nbd_lat = new_df.loc[i, 'latitude']
        nbd_lng = new_df.loc[i, 'longitude']

        radius = 1000 # Setting the radius as 1000 metres
        LIMIT = 30 # Getting the top 30 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nbd_name, nbd_lat, nbd_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

/Users/jordankupec/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [148]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Bridesburg-Kensington-Richmond,39.999581,-75.069072,Renzi's Pizzeria,Pizza Place,39.998184,-75.074312
1,Bridesburg-Kensington-Richmond,39.999581,-75.069072,Robeks Fresh Juices & Smoothies,Food Truck,40.001532,-75.075977
2,Bridesburg-Kensington-Richmond,39.999581,-75.069072,Wawa,Convenience Store,39.996568,-75.074419
3,Bridesburg-Kensington-Richmond,39.999581,-75.069072,Rite Aid,Pharmacy,39.997410,-75.074028
4,Bridesburg-Kensington-Richmond,39.999581,-75.069072,Edgemont Caterers,Food & Drink Shop,39.998726,-75.074137


In [151]:
explore_df.tail()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
12663,Yorktown,39.974579,-75.155997,Dunkin',Donut Shop,39.969383,-75.159455
12664,Yorktown,39.974579,-75.155997,The Maas Building,Music Venue,39.972063,-75.145460
12665,Yorktown,39.974579,-75.155997,The Bagel Hut,Bagel Shop,39.979942,-75.156317
12666,Yorktown,39.974579,-75.155997,Lorraine,Bar,39.968080,-75.162245
12667,Yorktown,39.974579,-75.155997,Amasi,Middle Eastern Restaurant,39.979562,-75.163875


In [152]:
philadelphia_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")

In [154]:
philadelphia_onehot['Neighborhood'] = explore_df['Neighbourhood']

In [155]:
philadelphia_onehot

,Accessories Store,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bridesburg-Kensington-Richmond
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bridesburg-Kensington-Richmond
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bridesburg-Kensington-Richmond
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bridesburg-Kensington-Richmond
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bridesburg-Kensington-Richmond
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Yorktown
12664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Yorktown
12665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Yorktown
12666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Yorktown


In [156]:
fixed_columns = [philadelphia_onehot.columns[-1]] + philadelphia_onehot.columns[:-1].values.tolist()
philadelphia_onehot = philadelphia_onehot[fixed_columns]

In [157]:
philadelphia_onehot

,Neighborhood,Accessories Store,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Bridesburg-Kensington-Richmond,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bridesburg-Kensington-Richmond,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bridesburg-Kensington-Richmond,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bridesburg-Kensington-Richmond,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bridesburg-Kensington-Richmond,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,Yorktown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12664,Yorktown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12665,Yorktown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12666,Yorktown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
philadelphia_grouped = philadelphia_onehot.groupby('Neighborhood').mean().reset_index()
philadelphia_grouped

,Neighborhood,Accessories Store,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Academy Gardens,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,Allegheny West,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.0
2,Andorra,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,Angora,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,Ashton-Woodenbridge,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Wister,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
185,Woodland Terrace,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.0
186,Wynnefield,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
187,Wynnefield Heights,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [161]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [180]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = philadelphia_grouped['Neighborhood']

for ind in np.arange(philadelphia_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(philadelphia_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Academy Gardens,Golf Course,Pizza Place,Discount Store,Pharmacy,Gym / Fitness Center,Deli / Bodega,Liquor Store,Sporting Goods Shop,Basketball Stadium,Playground
1,Allegheny West,Gym / Fitness Center,Intersection,Grocery Store,Fast Food Restaurant,Sandwich Place,Massage Studio,Liquor Store,Pizza Place,Climbing Gym,Food & Drink Shop
2,Andorra,Ice Cream Shop,Italian Restaurant,Gas Station,Laundromat,Trail,Dry Cleaner,Asian Restaurant,Playground,Bar,Disc Golf
3,Angora,Intersection,Chinese Restaurant,Pharmacy,Electronics Store,Sandwich Place,Shopping Plaza,Donut Shop,Park,Train Station,Snack Place
4,Ashton-Woodenbridge,Pizza Place,Mexican Restaurant,Grocery Store,Sandwich Place,Indian Restaurant,North Indian Restaurant,Martial Arts School,Massage Studio,Frozen Yogurt Shop,Men's Store


In [183]:
philadelphia_grouped_clustering = philadelphia_grouped.drop('Neighborhood', 1)

In [184]:
philadelphia_grouped_clustering

,Accessories Store,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
185,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.0
186,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
187,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [185]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot(x, y, xlabel, ylabel):
    plt.figure(figsize=(20,10))
    plt.plot(np.arange(2, x), y, 'o-')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(np.arange(2, x))
    plt.show()

In [186]:
max_range = 20

In [187]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [189]:
opt = np.argmax(scores) + 2 # Finds the optimal value
opt

2

In [233]:
kclusters = 5

# Run k-means clustering
pgc = philadelphia_grouped_clustering
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', random_state = 0).fit(pgc)

In [234]:
neighbourhoods_venues_sorted.drop(['Cluster Labels'], axis = 1, inplace = True)
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [235]:
philadelphia_merged = new_df
philadelphia_merged = philadelphia_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')
philadelphia_merged.dropna(inplace = True)
philadelphia_merged['Cluster Labels'] = philadelphia_merged['Cluster Labels'].astype(int)
philadelphia_merged.head()

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bridesburg-Kensington-Richmond,39.999581,-75.069072,1,Convenience Store,Pizza Place,Intersection,Sports Bar,Gym / Fitness Center,Pharmacy,Restaurant,Clothing Store,Bus Station,Food & Drink Shop
1,Bridesburg,39.999581,-75.069072,1,Convenience Store,Pizza Place,Intersection,Sports Bar,Gym / Fitness Center,Pharmacy,Restaurant,Clothing Store,Bus Station,Food & Drink Shop
2,Callowhill,39.959507,-75.155997,3,Park,Chinese Restaurant,Dim Sum Restaurant,Poke Place,Malay Restaurant,Rock Club,Latin American Restaurant,Gastropub,Tea Room,Noodle House
3,Chestnut Hill,40.070333,-75.207064,3,Bakery,American Restaurant,Grocery Store,Chinese Restaurant,Ice Cream Shop,Coffee Shop,Japanese Restaurant,Cheese Shop,Thai Restaurant,Seafood Restaurant
4,Chinatown,39.955205,-75.154278,3,Chinese Restaurant,Bubble Tea Shop,Ice Cream Shop,Vietnamese Restaurant,Bakery,Monument / Landmark,Shanghai Restaurant,Tea Room,Cajun / Creole Restaurant,Sandwich Place


In [236]:

# Create map
map_clusters = folium.Map(location=[phil_lat, phil_lng], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(philadelphia_merged['latitude'], philadelphia_merged['longitude'], philadelphia_merged['Neighborhood'], philadelphia_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster + 1) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

In [237]:
neighbourhoods_venues_sorted['Cluster Labels']

0      1
1      1
2      2
3      1
4      1
      ..
184    1
185    2
186    1
187    4
188    3
Name: Cluster Labels, Length: 189, dtype: int32